Objective
=========

To train a neural network to segment neurons, we first need data to train it on.
Our approach to this is to simulate imagery from a microscope using the geometry of digitized neuron models.
To do this, we source the models from [NeuroMorpho](https://neuromorpho.org) and load them into the microscope simulator.
This repository comes with a set of SWC files (the neuron models) from NeuroMorpho and can be used for this purpose.

### Get the File List

In [2]:
from pathlib import Path
from random import Random
import math
from PIL import Image
import neuroscope

def get_swc_list() -> list[Path]:
    entries = []
    for entry in Path('../data/swc').glob('*/*.swc'):
        entries.append(entry)
    return entries

swc_paths = get_swc_list()
print(f'Found {len(swc_paths)} files.')

Found 71 files.


### Split File List into Training and Validation Sets

In [4]:
import random

def train_val_split(paths: list[Path], train_ratio: float = 0.8, seed: int = 0) -> tuple[list[Path], list[Path]]:
    if not 0.0 < train_ratio < 1.0:
        raise ValueError(f"train_ratio must be between 0 and 1; got {train_ratio!r}")
    rng = random.Random(seed)
    shuffled = paths.copy()
    rng.shuffle(shuffled)
    split_idx = int(len(shuffled) * train_ratio)
    train_paths = shuffled[:split_idx]
    val_paths   = shuffled[split_idx:]
    return train_paths, val_paths

train_swc_paths, val_swc_paths = train_val_split(swc_paths)
print(f'Split data into {len(train_swc_paths)} training samples and {len(val_swc_paths)} validation samples.')

Split data into 56 training samples and 15 validation samples.


In [5]:

def open_models(swc_list: list[Path]) -> list[neuroscope.SWCModel]:
    models: list[neuroscope.SWCModel] = []
    for swc_path in swc_list:
        model = neuroscope.SWCModel()
        if not model.load_from_file(str(swc_path)):
            print(f'failed to open {swc_path}, skipping.')
            pass
        models.append(model)
    return models

def render_models(swc_models: list[neuroscope.SWCModel], out_dir: Path, res: tuple[int, int] = (512, 512), fov=600, seed: int = 0):
    out_dir.mkdir(exist_ok=True)
    seg_scope = neuroscope.SegmentationMicroscope(res[0], res[1], vertical_fov=fov)
    scope = neuroscope.FluorescenceMicroscope(res[0], res[1], vertical_fov=fov)
    counter = 0
    num_augmentations = 40
    rng = Random(seed)
    for model in swc_models:
        for _ in range(num_augmentations):

            tissue_config = neuroscope.TissueConfig()
            tissue_config.coverage = rng.uniform(0.6, 1.0)
            tissue_config.max_density = rng.uniform(0.01, 0.08)
            tissue_config.seed = rng.randint(0, 1000)

            tissue = neuroscope.Tissue()
            tissue.set_config(tissue_config)

            transform = neuroscope.Transform()
            transform.position = neuroscope.Vec3f(rng.uniform(-100, 100), rng.uniform(-100, 100), 0)
            transform.rotation = neuroscope.Vec3f(0, 0, rng.uniform(-math.pi, math.pi))

            # segmentation
            seg_scope.capture(model, tissue, transform)
            seg_buffer = seg_scope.copy_rgb_buffer()
            seg_path = out_dir / f'{counter:05}_mask.png'
            Image.frombytes('RGB', res, seg_buffer).save(seg_path)

            # real
            scope.capture(model, tissue, transform)
            real_buffer = scope.copy_buffer()
            real_path = out_dir / f'{counter:05}.png'
            Image.frombytes('L', res, real_buffer).save(real_path)

            counter += 1

out_dir = Path('../data/imagery')
out_dir.mkdir(exist_ok=True)
print('Starting rendering process. This may take a while.')
render_models(open_models(train_swc_paths), out_dir / 'train')
print('Training samples done rendering.')
render_models(open_models(val_swc_paths), out_dir / 'val')
print('Validation samples done rendering.')

Starting rendering process. This may take a while.
Training samples done rendering.
Validation samples done rendering.
